In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

In [3]:
print(keras.__version__)

2.15.0


In [17]:
!pip install cleanlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.0/334.0 kB 5.0 MB/s eta 0:00:00


In [20]:
import cleanlab
import numpy as np
import json
#from util import ALL_CLASSES
from urllib.request import urlopen
from skimage import io
from matplotlib import pyplot as plt

rc = {"axes.spines.left" : False,
      "axes.spines.right" : False,
      "axes.spines.bottom" : False,
      "axes.spines.top" : False,
      "xtick.bottom" : False,
      "xtick.labelbottom" : False,
      "ytick.labelleft" : False,
      "ytick.left" : False}
plt.rcParams.update(rc)

In [21]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [22]:
X_train = x_train.reshape(-1,32, 32, 3)  # reshaping for convnet
X_test = x_test.reshape(-1,32,32,3)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [23]:
print(X_train.shape)

(50000, 32, 32, 3)


In [24]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print(X_train.shape)

(40000, 32, 32, 3)


In [25]:
y_train

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [40]:
from tensorflow.keras.optimizers import Adam
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        #model = self.hypermodel.build(trial.hyperparameters)
        learning_rate = trial.hyperparameters.Float('learning_rate', 1e-4, 1e-2, sampling='log')
        dropout_rate = trial.hyperparameters.Float('dropout_rate', 0.0, 0.5)
        batch_size = trial.hyperparameters.Choice('batch_size', [16, 32])

        model = self.hypermodel.build(trial.hyperparameters)
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer = optimizer,loss='categorical_crossentropy',
        metrics=['accuracy'])
        model.fit(X, y,
                batch_size=batch_size ,
                **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

In [41]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))
    model.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu'))
    model.add(tf.keras.layers.AveragePooling2D((2,2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
    model.add(tf.keras.layers.AveragePooling2D((2,2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout_rate', min_value=0.0, max_value=0.5)))
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    # for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    #           model.add(tf.keras.layers.Conv2D(hp.Int(
    #               'units_{i}'.format(i=i), min_value=32, max_value=128, step=5), (3,3),activation='relu'))
    # model.add(tf.keras.layers.Flatten())
    # for i in range(hp.Int('n_connections', 1, 3)):
    #     model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',
    #                               values=[32,64,128, 256]), activation = 'relu'))
    # model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))

    return model


In [42]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')


my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3)


Trial 10 Complete [00h 04m 29s]
accuracy: 0.6259999871253967

Best accuracy So Far: 0.6679999828338623
Total elapsed time: 00h 42m 35s


In [43]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_48 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 average_pooling2d_48 (Aver  (None, 15, 15, 32)        0         
 agePooling2D)                                                   
                                                                 
 batch_normalization_48 (Ba  (None, 15, 15, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_49 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 average_pooling2d_49 (Aver  (None, 6, 6, 64)          0         
 agePooling2D)                                                   
                                                     

In [44]:
path = '/content/drive/MyDrive/mlops/model'
model.save(path)

In [47]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [48]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)


{'serving_default': {'inputs': ['inputs'], 'outputs': ['outputs']}}


In [49]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

In [50]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)